<a href="https://colab.research.google.com/github/SumiranRai/MDSC-Lab/blob/main/MDSC-203-Machine-Learning/Machine_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning Assignment

By Sumiran Rai

Regd. No. - 24040208007

### Understanding the Data

Importing required libraries

In [1]:
.import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

SyntaxError: invalid syntax (<ipython-input-1-1517b9dcce76>, line 1)

Loading the data

In [2]:
data = pd.read_csv('/content/sample_data/diabetes_012_health_indicators_BRFSS2015.csv')
data.head()

NameError: name 'pd' is not defined

Shape of the data

In [ ]:
print(data.shape)

Data type of each feature

In [ ]:
types = data.dtypes
print(types)

Let's understand the data

In [ ]:
from pandas import set_option
with pd.option_context('display.width', 150, 'display.precision', 2):
  description = data.describe()
  print(description)

###Dataset is Imbalanced!

0 - No Diabetes

1 - Pre-Diabetic

2 - Diabetic

In [ ]:
class_counts = data.groupby('Diabetes_012').size()
print(class_counts)

Correlations

In [ ]:
with pd.option_context('display.width', 150, 'display.precision', 2):
  correlations = data.corr(method='pearson')
  print(correlations)

Skewness

In [ ]:
skew = data.skew()
print(skew)

### Visualizing the Data

#### Histograms

In [ ]:
# Create histograms
fig = data.hist(figsize=(10, 6), bins=20, grid=True)

# Adjust layout and font sizes
plt.tight_layout()
plt.subplots_adjust(hspace=1, wspace=1)  # Adjust spacing between subplots

# Reduce font size for better readability
for ax in fig.flatten():
    ax.set_xlabel(ax.get_xlabel(), fontsize=10)
    ax.set_ylabel(ax.get_ylabel(), fontsize=10)
    ax.set_title(ax.get_title(), fontsize=12)
    ax.tick_params(axis='both', labelsize=8)  # Reduce tick label font size

plt.show()

#### Density Plots

In [ ]:
#data.plot(kind='density', subplots=True, layout=(5,5), sharex=False)
#plt.show()

Box and Whisker Plots

In [ ]:
axes = data.plot(kind='box', subplots=True, layout=(5, 5), sharex=False, sharey=False, figsize=(12, 10), patch_artist=True, vert=True)

plt.tight_layout()
plt.subplots_adjust(hspace=0.5, wspace=0.5)

if isinstance(axes, np.ndarray):  # If axes is an array
    axes = axes.flatten()
else:  # If only one subplot exists (Series case)
    axes = [axes]

plt.show()

#### Correlation matrix

In [ ]:
# Plot correlation matrix
fig, ax = plt.subplots(figsize=(8, 6))
cax = ax.matshow(correlations, vmin=-1, vmax=1, cmap='coolwarm')
fig.colorbar(cax)

# Set ticks and labels
ticks = np.arange(len(data.columns))
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_xticklabels(data.columns, rotation=90)  # Rotate for better readability
ax.set_yticklabels(data.columns)

plt.show()

Generic Correlation Matrix

In [ ]:
#plot correlation matrix
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1, cmap='coolwarm')
fig.colorbar(cax)
plt.show()


#### Scatter Plot

from pandas.plotting import scatter_matrix  

fig = scatter_matrix(data, figsize=(10, 10), diagonal='hist', marker='o', alpha=0.7, s=10, hist_kwds={'bins': 20})

for ax in fig.flatten():
    ax.xaxis.label.set_fontsize(10)
    ax.yaxis.label.set_fontsize(10)
    ax.tick_params(axis='both', labelsize=8)

plt.xticks(rotation=45)
plt.show()

### Data Preparation

#### Rescale Data

In [ ]:
from numpy import set_printoptions
from sklearn.preprocessing import MinMaxScaler

array = data.values

# First column (index 0) is the target (Y)
Y = array[:, 0]

# Remaining 21 columns (index 1 to 21) are features (X)
X = array[:, 1:22]

# Apply Min-Max Scaling to features only
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX = scaler.fit_transform(X)

# Print first 5 rows of scaled data
set_printoptions(precision=3)
print(rescaledX[0:5, :])

#### Standardize Data

In [ ]:
from sklearn.preprocessing import StandardScaler

array = data.values

# separate array into input and output components
X = array[:, 1:22]
Y = array[:, 0]
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)
# summarize transformed data
set_printoptions(precision=3)
print(rescaledX[0:5,:])

#### Normalize Data

In [ ]:
from sklearn.preprocessing import Normalizer

array = data.values
# separate array into input and output components
X = array[:, 1:22]
Y = array[:, 0]
scaler = Normalizer().fit(X)
normalizedX = scaler.transform(X)
# summarize transformed data
set_printoptions(precision=3)
print(normalizedX[0:5,:])

#### Binarize Data

In [ ]:
from sklearn.preprocessing import Binarizer

array = data.values
# separate array into input and output components
X = array[:,1:22]
Y = array[:,0]
binarizer = Binarizer(threshold=0.0).fit(X)
binaryX = binarizer.transform(X)
# summarize transformed data
set_printoptions(precision=3)
print(binaryX[0:5,:])

### Feature Selection

#### Univariate Selection

Statistical tests can be used to select those features that have the strongest relationship with
the output variable.
The scikit-learn library provides the SelectKBest class2
that can be used
with a suite of different statistical tests to select a specific number of features. The example
below uses the chi-squared (chi2
) statistical test for non-negative features to select 11 of the best
features from the dataset.

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
import pandas as pd
import numpy as np

# Feature selection
X = data.iloc[:, 1:22]  # Selecting feature columns
Y = data.iloc[:, 0]     # Target column

# Applying SelectKBest
test = SelectKBest(score_func=chi2, k=11)
fit = test.fit(X, Y)

# Get selected feature indices
selected_indices = fit.get_support(indices=True)

# Get the feature names
selected_features = X.columns[selected_indices]

# Print scores and selected feature names
print("Feature Scores:\n", fit.scores_)
print("\nSelected Features:\n", selected_features)

# Transform dataset to include only selected features
X_selected = fit.transform(X)

# Convert to DataFrame for better readability
X_selected_df = pd.DataFrame(X_selected, columns=selected_features)

# Display first 5 rows of selected features
print(X_selected_df.head())

#### Recursive Feature Elimination

The Recursive Feature Elimination (or RFE) works by recursively removing attributes and
building a model on those attributes that remain. It uses the model accuracy to identify which attributes (and combination of attributes) contribute the most to predicting the target attribute.

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pandas as pd

# Convert NumPy array to DataFrame (if necessary)
if isinstance(data, pd.DataFrame):  # If data is already a DataFrame
    X = data.iloc[:, 1:22]  # Select feature columns
    Y = data.iloc[:, 0]  # Target variable
    feature_names = X.columns  # Get column names
else:  # If data is a NumPy array
    feature_names = [f"Feature_{i}" for i in range(1, 22)]  # Create dummy labels
    X = pd.DataFrame(data[:, 1:22], columns=feature_names)  # Convert to DataFrame
    Y = pd.Series(data[:, 0])  # Convert target variable to a Pandas Series

# Define model
model = LogisticRegression(solver='liblinear')

# Apply RFE
num_features = 7  # Change as needed
rfe = RFE(estimator=model, n_features_to_select=num_features)
rfe.fit(X, Y)

# Get selected feature names
selected_features = X.columns[rfe.support_]

# Print results
print(f"Selected {num_features} Features:", list(selected_features))
print("Feature Ranking:", rfe.ranking_)

# Create DataFrame with selected features
X_selected_df = X.loc[:, rfe.support_]
print(X_selected_df.head())  # Display first 5 rows


#### Principal Component Analysis

Principal Component Analysis (or PCA) uses linear algebra to transform the dataset into a
compressed form. Generally this is called a data reduction technique. A property of PCA is that
you can choose the number of dimensions or principal components in the transformed result.

PCA with Feature Labels

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Convert NumPy array to DataFrame if necessary
if isinstance(data, pd.DataFrame):  # If data is already a DataFrame
    X = data.iloc[:, 0:8]  # Select feature columns
    feature_names = X.columns  # Get column names
else:  # If data is a NumPy array
    feature_names = [f"Feature_{i}" for i in range(8)]  # Create dummy labels
    X = pd.DataFrame(data[:, 0:8], columns=feature_names)  # Convert to DataFrame

# Apply PCA
n_components = 1 # Change as needed
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

# Create a DataFrame with PCA components
pca_df = pd.DataFrame(X_pca, columns=[f'PC{i+1}' for i in range(n_components)])

# Print explained variance ratio
print("Explained Variance Ratio:", pca.explained_variance_ratio_)

# Get feature contributions to principal components
pca_components = pd.DataFrame(pca.components_, columns=feature_names,
                              index=[f'PC{i+1}' for i in range(n_components)])
print("\nFeature Contributions to Each Principal Component:\n", pca_components)

# Show the transformed PCA dataset
print("\nFirst 5 rows of transformed PCA features:\n", pca_df.head())

Determine the Optimal Number of Components

In [ ]:
pca_full = PCA().fit(X)  # Fit PCA with all components
explained_variance = np.cumsum(pca_full.explained_variance_ratio_)

plt.plot(range(1, len(explained_variance)+1), explained_variance, marker='o', linestyle='--')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Scree Plot')
plt.grid()
plt.show()

 Automatic Selection (95% Variance)

 If the optimal number of principal components (PCs) is 1, it means that a single principal component explains most of the variance in your dataset.

In [ ]:
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
optimal_components = np.argmax(cumulative_variance >= 0.95) + 1  # First index where ≥ 95%
print(f"Optimal number of components: {optimal_components}")


#### Feature Importance

Bagged decision trees like Random Forest and Extra Trees can be used to estimate the importance
of features.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier

# Ensure `X` is a DataFrame (if `data` is available)
if isinstance(data, pd.DataFrame):
    X = data.iloc[:, 1:22]  # Feature columns
    Y = data.iloc[:, 0]  # Target variable
    feature_names = X.columns  # Get feature labels
else:
    feature_names = [f"Feature_{i}" for i in range(1, 22)]  # Create dummy labels
    X = pd.DataFrame(array[:, 1:22], columns=feature_names)
    Y = pd.Series(array[:, 0])  # Ensure Y is a Series

# Feature importance with Extra Trees Classifier
model = ExtraTreesClassifier(n_estimators=100, random_state=42)  # Use more trees for better stability
model.fit(X, Y)

# Get feature importance scores
feature_importance = model.feature_importances_

# Create DataFrame for better visualization
importance_df = pd.DataFrame({"Feature": feature_names, "Importance": feature_importance})
importance_df = importance_df.sort_values(by="Importance", ascending=False)  # Sort by importance

# Print sorted feature importance
print(importance_df)

# Optional: Plot the feature importance
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.barh(importance_df["Feature"], importance_df["Importance"], color='skyblue')
plt.xlabel("Feature Importance Score")
plt.ylabel("Feature")
plt.title("Feature Importance using Extra Trees Classifier")
plt.gca().invert_yaxis()  # Reverse order for better visualization
plt.show()

### Performance Evaluation of ML Algorithms

#### Split into Train and Test Sets

70% - Training set

30% - Test set

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import ConvergenceWarning
import warnings

# Ignore convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Split dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=7)

# Feature Scaling (Standardization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define & Train Logistic Regression
model = LogisticRegression(solver='saga', max_iter=1000, C=1.0)  # C=1.0 (default regularization)
model.fit(X_train, Y_train)

# Evaluate Model
accuracy = model.score(X_test, Y_test) * 100.0
print(f"Accuracy: {accuracy:.3f}%")


#### K-fold Cross Validation

K-Fold Cross Validation is a technique to estimate model performance with lower variance than a single train-test split. It works by dividing the dataset into k equal parts (folds), where the model is trained on k-1 folds and tested on the remaining one. This process repeats k times, with each fold serving as the test set once.

After training, we get k performance scores, which are averaged to provide a more reliable estimate of the model's performance. Common values of k are 3, 5, or 10, depending on dataset size. This method ensures the model is tested on different data, leading to a more generalizable evaluation.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# Define k-fold cross-validation
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)  # Fixed issue

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Scale the entire dataset

# Define model
model = LogisticRegression(solver='saga', max_iter=1000)  # saga is better for large datasets

# Evaluate model using cross-validation
results = cross_val_score(model, X_scaled, Y, cv=kfold, scoring='accuracy')

# Print results
print(f"Accuracy: {results.mean() * 100:.3f}% ± {results.std() * 100:.3f}%")

#### Leave One Out Cross Validation

This variation of cross validation is called leave-one-out cross
validation. The result is a large number of performance measures that can be summarized in an effort to give a more reasonable estimate of the accuracy of your model on unseen data.

A downside is that it can be a computationally more expensive procedure than k-fold cross
validation.

Leave-One-Out (LOO) is computationally expensive → Not ideal for large datasets (250,000 instances).

In [ ]:
"""
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Scale the entire dataset

# Define Leave-One-Out Cross-Validation
loocv = LeaveOneOut()

# Define Model
model = LogisticRegression(solver='saga', max_iter=1000)  # 'saga' is better for large datasets

# Evaluate Model
results = cross_val_score(model, X_scaled, Y, cv=loocv, scoring='accuracy')

# Print Results
print(f"Accuracy: {results.mean() * 100:.3f}% ± {results.std() * 100:.3f}%")
"""

# Skipped evaluating because it took too much time.

#### Repeated Random Test-Train Splits

Repeated Random Train-Test Splitting is a variation of k-fold cross-validation that repeatedly splits the dataset into training and testing sets. It offers:

✔ Faster evaluation than k-fold cross-validation.

✔ Lower variance than a single train-test split.

✔ More repetitions for improved accuracy.

🔹 Downside: Some data points may appear in multiple test sets, causing redundancy.


In [ ]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define Shuffle Split CV
n_splits = 10
test_size = 0.33
seed = 7
shuffle_split = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)

# Define Model
model = LogisticRegression(solver='saga', max_iter=1000)  # 'saga' works well for large datasets

# Evaluate Model
results = cross_val_score(model, X_scaled, Y, cv=shuffle_split, scoring='accuracy')

# Print Results
print(f"Accuracy: {results.mean() * 100:.3f}% ± {results.std() * 100:.3f}%")


### ML Algorithm Performance Metrics

#### Classification Accuracy

Classification accuracy is the number of correct predictions made as a ratio of all predictions
made. This is the most common evaluation metric for classification problems, it is also the most
misused. It is really only suitable when there are an equal number of observations in each class
(which is rarely the case) and that all predictions and prediction errors are equally important,
which is often not the case.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import numpy as np

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define K-Fold CV
kfold = KFold(n_splits=10, random_state=7, shuffle=True)  # Added shuffle for randomness

# Define Model
model = LogisticRegression(solver='saga', max_iter=1000)  # 'saga' for large datasets

# Evaluate Model
results = cross_val_score(model, X_scaled, Y, cv=kfold, scoring='accuracy')

# Print Results
print(f"Accuracy: {results.mean():.3f} ± {results.std():.3f}")

#### Logatrithmic Loss

Logarithmic loss (or logloss) is a performance metric for evaluating the predictions of probabilities
of membership to a given class. The scalar probability between 0 and 1 can be seen as a measure
of confidence for a prediction by an algorithm. Predictions that are correct or incorrect are
rewarded or punished proportionally to the confidence of the prediction.

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming 'data' is your DataFrame
X = data.iloc[:, 1:22]  # Select feature columns (adjusted for 22 features)
Y = data.iloc[:, 0]  # Target column

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define cross-validation strategy
kfold = KFold(n_splits=10, random_state=7, shuffle=True)  # Shuffling for better results

# Define model
model = LogisticRegression(solver='saga', max_iter=500)  # 'saga' works well for large datasets

# Use Log Loss as the evaluation metric
scoring = 'neg_log_loss'
results = cross_val_score(model, X_scaled, Y, cv=kfold, scoring=scoring)

# Print log loss correctly
print(f"Log Loss: {results.mean():.3f} ({results.std():.3f})")


### Area Under ROC Curve

Area under ROC Curve (or AUC for short) is a performance metric for binary classification
problems. The AUC represents a model’s ability to discriminate between positive and negative
classes.

An area of 1.0 represents a model that made all predictions perfectly. An area of
0.5 represents a model that is as good as random.


In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming 'data' is your DataFrame
X = data.iloc[:, 1:22].values  # Select feature columns
Y = data.iloc[:, 0].values  # Target column

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define cross-validation strategy
kfold = KFold(n_splits=10, random_state=7, shuffle=True)

# Define model with multi-class support
model = OneVsRestClassifier(LogisticRegression(solver='saga', max_iter=500))  # or 'multinomial'

# Use ROC AUC for multi-class classification
scoring = 'roc_auc_ovr'  # Use 'roc_auc_ovo' for one-vs-one
results = cross_val_score(model, X_scaled, Y, cv=kfold, scoring=scoring)

# Print ROC AUC correctly
print(f"AUC: {results.mean():.3f} ({results.std():.3f})")

#### Confusion Matrix

A confusion matrix summarizes model performance by comparing predicted vs. actual class labels. Rows represent actual classes, columns represent predictions, and cells show correct and incorrect counts. It helps evaluate classification accuracy for both binary and multi-class problems.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

# Assuming 'data' is your DataFrame
X = data.iloc[:, 1:22].values  # Feature columns
Y = data.iloc[:, 0].values  # Target column (multi-class)

# Train-test split
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

# Define logistic regression model with multi-class support
model =
model = OneVsRestClassifier(LogisticRegression(solver='saga', max_iter=500) ) # or 'multinomial'
model.fit(X_train, Y_train)

# Make predictions
predicted = model.predict(X_test)

# Compute confusion matrix
matrix = confusion_matrix(Y_test, predicted)

# Print the confusion matrix
print("Confusion Matrix:")
print(matrix)
